In [1]:
import cv2
import pytesseract
import re
import json
from http.server import BaseHTTPRequestHandler, HTTPServer
import cgi
import logging
import sys


In [2]:
def fetch_form_data_list(fileName):
    card_list=[]
    cust_list=[]
    cardField=False
    custField=False
    with open(fileName, 'r') as file:
        for line in file:
            line = line.strip() 
            if line == "":
                continue
                
            if "[CardField]" in line:
                cardField=True
                custField=False                
                continue
            
            if "[CustomerField]" in line:
                cardField=False
                custField=True                
                continue
                
            if cardField:
                card_list.append(line) 
            elif custField:
                cust_list.append(line)
               
      
    return card_list,cust_list

In [3]:
def process_image(fileName,formType):
    img = cv2.imread('./template/'+fileName)
    imgStr = pytesseract.image_to_string(img,config='--psm 4')
    # form data to add the key words
    #formDataList=["Naam","Titel","Bevoegdheid","Datum in functie"]
    fileTypeDict={'reg':'formdata.props','fed':'formdata.props'}   
    filePath="./config/"+fileTypeDict[formType]
    cardDataList,custDataList= fetch_form_data_list(filePath)
    #formDataList=["Naam","Titel"]
    dataList=[]
    dataDict={}
    custInfoDict={}
    # process the text and filter out the form data key values
    for line in imgStr.split('\n'): 
        for eachElement in cardDataList:        
            if eachElement in line:
                keyVal = line.split(eachElement)            
                if eachElement in dataDict.keys():
                    dataList.append(dataDict)
                    dataDict={}
                val = mystring = re.sub(r"^\W+", "", keyVal[1])              
                dataDict[eachElement]=val    
        for eachElement in custDataList:
            if eachElement in line:                
                keyVal = line.split(eachElement)                
                val = re.sub(r"^\W+", "", keyVal[1])
                custInfoDict[eachElement]=val               

    dataList.append(dataDict)
    # convert the data list to json format    
    json_dict={}
    json_dict['card_info'] = dataList
    json_dict['cust_info'] = custInfoDict
    json_data = json.dumps(json_dict)
       
    return json_data

In [4]:
class RequestHandler(BaseHTTPRequestHandler):
    def do_OPTIONS(self):
        self.send_response(200)
        self.send_header('Content-type', 'application/json')
        self.send_header("Cache-Control", "no-cache")
        self.send_header("Access-Control-Allow-Origin", "http://localhost:3000")
        self.send_header("Access-Control-Allow-Methods", "POST, GET")
        self.send_header("Access-Control-Allow-Headers", "Origin, X-Requested-With, Content-Type, Accept, Authorization,Access-Control-Allow-Origin")
        
        self.end_headers()
        return
    
    def do_GET(self):        
        self.send_response(200)
        self.send_header('Content-type', 'application/json')
        self.send_header("Cache-Control", "no-cache")
        self.send_header("Access-Control-Allow-Origin", "http://localhost:3000")
        self.send_header("Access-Control-Allow-Methods", "POST, GET")
        self.send_header("Access-Control-Allow-Headers", "Origin, X-Requested-With, Content-Type, Accept, Authorization,Access-Control-Allow-Origin")
        
        self.end_headers()
        output = '{"result":"success"}'
        self.wfile.write(output.encode('utf-8'))
        return
    
    def do_POST(self):  
        try:
            '''Handle POST requests.'''
            logging.debug('POST %s' % (self.path))  
            ctype, pdict = cgi.parse_header(self.headers['content-type'])
            if ctype =='application/json':
                content_length = cgi.parse_header(self.headers['content-length'])
                print(content_length[0])
                length = int(content_length[0])                
                post_body = self.rfile.read(length)
                data = json.loads(post_body.decode('utf-8'))
                imageName=data['imageName']
                imageType=data['imageType']
                print(imageName)
                print(imageType)
                output = process_image(imageName,imageType)             
                self.send_response(200)
                self.send_header('Content-type', 'application/json')
                self.send_header("Access-Control-Allow-Origin", "http://localhost:3000")
                self.send_header("Access-Control-Allow-Methods", "POST, GET")
                self.send_header("Access-Control-Allow-Headers", "Origin, X-Requested-With, Content-Type, Accept, Authorization,Access-Control-Allow-Origin")
                self.end_headers()            
                self.wfile.write(output.encode('utf-8'))  
                print(output)
            return            
        except:
            self.send_error(404, "{}".format(sys.exc_info()[0]))
            
            self.send_header("Access-Control-Allow-Origin", "http://localhost:3000")
            self.send_header("Access-Control-Allow-Methods", "POST, GET")
            self.send_header("Access-Control-Allow-Headers", "Origin, X-Requested-With, Content-Type, Accept, Authorization,Access-Control-Allow-Origin")

            self.end_headers()   
            print(sys.exc_info())     
   



In [ ]:
if __name__ == "__main__":
    server_address = ('localhost',8880)
    httpd = HTTPServer(server_address, RequestHandler)
    print('Listening on port 8880 ...')
    httpd.serve_forever()

Listening on port 8880 ...
69
uittreksel-KVK-edit-enhance (2).jpg
reg


127.0.0.1 - - [10/Oct/2018 22:17:43] "POST / HTTP/1.1" 200 -


{"card_info": [{"Naam": "Al Sud, Mana", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Voorzitter", "Bevoegdheid": "Gezameniijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "Shayea - Al-Khamas, Nidyal Shakir", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Penningmeester", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "ALSabti, Rafid Abdulla Najim", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)"}, {"Naam": "Badri, Hamed"}], "cust_info": {"KvK-nummer": "59992298", "RSIN": "953722791", "Rechtsvorm": "Vereniging met volledige rechtsbevoegdheid", "Bezoekadres": "Ruimtevaartbaan 31, 34020K Usselstein", "Telefoonnummer": "0644207733"}}
69
uittreksel-KVK-edit-enhance (2).jpg
reg


127.0.0.1 - - [10/Oct/2018 22:19:31] "POST / HTTP/1.1" 200 -


{"card_info": [{"Naam": "Al Sud, Mana", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Voorzitter", "Bevoegdheid": "Gezameniijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "Shayea - Al-Khamas, Nidyal Shakir", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Penningmeester", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "ALSabti, Rafid Abdulla Najim", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)"}, {"Naam": "Badri, Hamed"}], "cust_info": {"KvK-nummer": "59992298", "RSIN": "953722791", "Rechtsvorm": "Vereniging met volledige rechtsbevoegdheid", "Bezoekadres": "Ruimtevaartbaan 31, 34020K Usselstein", "Telefoonnummer": "0644207733"}}
69
uittreksel-KVK-edit-enhance (2).jpg
reg


...

{"card_info": [{"Naam": "Al Sud, Mana", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Voorzitter", "Bevoegdheid": "Gezameniijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "Shayea - Al-Khamas, Nidyal Shakir", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Penningmeester", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "ALSabti, Rafid Abdulla Najim", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)"}, {"Naam": "Badri, Hamed"}], "cust_info": {"KvK-nummer": "59992298", "RSIN": "953722791", "Rechtsvorm": "Vereniging met volledige rechtsbevoegdheid", "Bezoekadres": "Ruimtevaartbaan 31, 34020K Usselstein", "Telefoonnummer": "0644207733"}}
69
uittreksel-KVK-edit-enhance (2).jpg
reg


127.0.0.1 - - [10/Oct/2018 22:25:25] "POST / HTTP/1.1" 200 -


{"card_info": [{"Naam": "Al Sud, Mana", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Voorzitter", "Bevoegdheid": "Gezameniijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "Shayea - Al-Khamas, Nidyal Shakir", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Penningmeester", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "ALSabti, Rafid Abdulla Najim", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)"}, {"Naam": "Badri, Hamed"}], "cust_info": {"KvK-nummer": "59992298", "RSIN": "953722791", "Rechtsvorm": "Vereniging met volledige rechtsbevoegdheid", "Bezoekadres": "Ruimtevaartbaan 31, 34020K Usselstein", "Telefoonnummer": "0644207733"}}
69
uittreksel-KVK-edit-enhance (2).jpg
reg


127.0.0.1 - - [10/Oct/2018 22:25:39] "POST / HTTP/1.1" 200 -


{"card_info": [{"Naam": "Al Sud, Mana", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Voorzitter", "Bevoegdheid": "Gezameniijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "Shayea - Al-Khamas, Nidyal Shakir", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)", "Titel": "Penningmeester", "Bevoegdheid": "Gezamenlijk bevoegd (met andere bestuurder(s), zie statuten)"}, {"Naam": "ALSabti, Rafid Abdulla Najim", "Datum in functie": "13-02-2014 (datum registratie: 13-02-2014)"}, {"Naam": "Badri, Hamed"}], "cust_info": {"KvK-nummer": "59992298", "RSIN": "953722791", "Rechtsvorm": "Vereniging met volledige rechtsbevoegdheid", "Bezoekadres": "Ruimtevaartbaan 31, 34020K Usselstein", "Telefoonnummer": "0644207733"}}
